In [1]:
import pandas as pd 
from datetime import datetime


df= pd.read_csv('data/dataELO.csv')
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df.tail(20)

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_6716\3007485351.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')


,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
22637,Série A,2023,2023-08-26,Corinthians,1,1,Goiás,E,1,1,200
22638,Série C,2023,2023-08-26,Ypiranga-RS,1,1,Botafogo-PB,E,1,1,188
22639,Série C,2023,2023-08-26,Remo,3,1,Altos,VC,3,0,187
22640,Série C,2023,2023-08-26,Operário-PR,1,0,Brusque,VC,3,0,186
22641,Série B,2023,2023-08-26,Tombense,2,2,Ceará,E,1,1,245
22642,Série B,2023,2023-08-26,Sampaio Corrêa,1,1,Guarani,E,1,1,246
22643,Série B,2023,2023-08-26,CRB,1,0,Novorizontino,VC,3,0,247
22644,Série A,2023,2023-08-26,Flamengo,0,0,Internacional,E,1,1,199
22645,Série C,2023,2023-08-26,Figueirense,0,0,Manaus,E,1,1,185
22646,Série C,2023,2023-08-26,Amazonas,1,0,CSA,VC,3,0,190


In [2]:
#This function updates the Elo ratings and creates a ranking

timestamp_df = pd.DataFrame(columns=['Timestamp'])
def update_elo(df):
    #Gives different weights to each competition
    k_factor_map = {
                    'Série C': 10, 
                    'Série B': 20, 
                    'CBR': 25, 
                    'Série A': 30, 
                    }
    teams = {}
    elo_ranking = {}
    average_elo = {}
    team_games = {}
    team_victories = {}
    team_losses = {}
    team_goals = {}
    team_draws = {}
    last_competition = {}
    current_year = datetime.now().year
    
    #Creates a list of teams and gives then an initial rating of 1500
    for index, row in df.iterrows():
        home_team, away_team = row['Casa'], row['Fora']
        comp_level = row['Comp']
        match_date = row['Data'].year
        k_factor = k_factor_map.get(comp_level, 50) 
        if home_team not in teams:
            teams[home_team] = 1500
            elo_ranking[home_team] = teams[home_team]
        if away_team not in teams:
            teams[away_team] = 1500
            elo_ranking[away_team] = teams[away_team]
        home_elo, away_elo = teams[home_team], teams[away_team]
        home_score, away_score = row['GC'], row['GF']
        result = row['Res']
        # Only update the last competition if it's not 'CBR' and the match is in the current year
        if comp_level != 'CBR' and match_date == current_year:
            last_competition[home_team] = comp_level
            last_competition[away_team] = comp_level

        #Gives different weights for home and away games
        if result == 'VC':
            home_result, away_result = 1, 0
        elif result == 'VV':
            home_result, away_result = 0, 1
        else:
            home_result, away_result = 0.5, 0.5
        home_expected = 1 / (1 + 10 ** ((away_elo - home_elo - 100) / 400))
        away_expected = 1 / (1 + 10 ** ((home_elo - away_elo + 100) / 400))
        
        #Calculates the number of goals
        if home_team in team_goals:
            team_goals[home_team] += home_score
        else:
            team_goals[home_team] = home_score

        if away_team in team_goals:
            team_goals[away_team] += away_score
        else:
            team_goals[away_team] = away_score
       # Update the number of victories, losses, and draws for each team
        if result == 'VC':
            home_result, away_result = 1, 0
            if home_team in team_victories:
                team_victories[home_team] += 1
            else:
                team_victories[home_team] = 1
            if away_team in team_losses:
                team_losses[away_team] += 1
            else:
                team_losses[away_team] = 1
        elif result == 'VV':
            home_result, away_result = 0, 1
            if home_team in team_losses:
                team_losses[home_team] += 1
            else:
                team_losses[home_team] = 1
            if away_team in team_victories:
                team_victories[away_team] += 1
            else:
                team_victories[away_team] = 1
        else:
            home_result, away_result = 0.5, 0.5
            if home_team in team_draws:
                team_draws[home_team] += 1
            else:
                team_draws[home_team] = 1
            if away_team in team_draws:
                team_draws[away_team] += 1
            else:
                team_draws[away_team] = 1
        # Update the number of games played for each team
        if home_team in team_games:
            team_games[home_team] += 1
        else:
            team_games[home_team] = 1
        if away_team in team_games:
            team_games[away_team] += 1
        else:
            team_games[away_team] = 1
        
        # Calculate the goal difference and adjust the K factor accordingly
        goal_diff = abs(home_score - away_score)
        if goal_diff >= 4:
            k_factor = k_factor + 0.75 + (goal_diff - 4) / 8
        elif goal_diff == 3:
            k_factor = k_factor + 0.75
        elif goal_diff == 2:
            k_factor = k_factor + 0.5
        else:
            k_factor = k_factor
            
        # Calculate the rating difference factor
        rating_diff = abs(home_elo - away_elo)
        if rating_diff > 400:
            rating_diff = 400
        rating_factor = rating_diff / 400 + 1
            
        teams[home_team] = home_elo + rating_factor * k_factor * (home_result - home_expected)
        teams[away_team] = away_elo + rating_factor * k_factor * (away_result - away_expected)
        teams[home_team] = round(teams[home_team], 2)
        teams[away_team] = round(teams[away_team], 2)
        elo_ranking[home_team] = teams[home_team]
        elo_ranking[away_team] = teams[away_team]
        df.at[index, 'Elo Casa'] = home_elo
        df.at[index, 'Elo Fora'] = away_elo
        df.at[index, 'Novo Elo Casa'] = teams[home_team]
        df.at[index, 'Novo Elo Fora'] = teams[away_team]

        
        for team, elo in teams.items():
            if team in average_elo:
                average_elo[team].append(elo)
            else:
                average_elo[team] = [elo]

    # Compute average Elo for each team
    for team, elo_list in average_elo.items():
        average_elo[team] = round(sum(elo_list) / len(elo_list), 2)

    #Createa the Datasets
    elo_df = pd.DataFrame.from_dict(elo_ranking, orient='index', columns=['Elo'])
    elo_df = elo_df.sort_values(by='Elo', ascending=False)
    elo_df.index.name = 'Time'
    elo_df.reset_index(inplace=True)
    #elo_df = elo_df[~elo_df['Time'].str.contains(r'\(ARG\)|\(BOL\)|\(CHI\)|\(COL\)|\(EQU\)|\(MEX\)|\(PAR\)|\(PER\)|\(URU\)|\(VEN\)', regex=True)]
    elo_df['Jogos'] = elo_df['Time'].map(team_games) 
    elo_df = elo_df[elo_df['Jogos'] >= 30]
    elo_df['Rank'] = elo_df['Elo'].rank(ascending=False, method='dense')
    elo_df['Média'] = elo_df['Time'].map(average_elo).round(2)
    elo_df['#'] = elo_df['Média'].rank(ascending=False, method='dense')
    elo_df['Gols'] = elo_df['Time'].map(team_goals)
    elo_df['Vitórias'] = elo_df['Time'].map(team_victories)
    elo_df['Derrotas'] = elo_df['Time'].map(team_losses)
    elo_df['Empates'] = elo_df['Time'].map(team_draws)
    elo_df['Série'] = elo_df.apply(lambda row: last_competition.get(row['Time'], '-'), axis=1) 
    elo_df.replace({'Série A': 'A','Série B': 'B','Série C': 'C',                   
            }, inplace=True)
    timestamp_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    timestamp_df.loc[len(timestamp_df)] = [timestamp_now]

    
    return df, elo_df

df, elo_df = update_elo(df)
df.to_csv('data/dataELOranking.csv', index=False)
elo_df.to_json('data/dataELOranking.json', orient='records')
timestamp_df.to_json('data/timestamps.json')



In [3]:
filt = (elo_df['Série'] == "A")
elo_df[filt]

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
0,Botafogo,1919.20,858,1.0,1667.25,12.0,1167,344,287,227,A
1,Palmeiras,1898.90,849,2.0,1734.05,2.0,1308,407,231,211,A
2,Flamengo,1877.06,885,3.0,1733.82,3.0,1306,409,241,235,A
3,Fluminense,1834.26,879,4.0,1691.46,10.0,1232,362,292,225,A
4,Fortaleza,1804.69,716,5.0,1586.32,24.0,964,280,245,191,A
5,Corinthians,1788.37,862,6.0,1715.66,5.0,1154,374,244,244,A
6,Red Bull Bragantino,1783.08,629,7.0,1572.76,27.0,812,241,226,162,A
7,Athletico-PR,1778.05,866,8.0,1675.52,11.0,1147,346,306,214,A
8,Internacional,1776.80,843,9.0,1713.55,6.0,1147,379,247,217,A
9,Atlético-MG,1775.87,872,10.0,1710.85,7.0,1334,388,272,212,A


In [4]:
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Elo Casa,Elo Fora,Novo Elo Casa,Novo Elo Fora
0,CBR,2003,2003-02-05,Anapolina,1,2,Náutico,VV,0,3,1,1500.00,1500.00,1484.00,1516.00
1,CBR,2003,2003-02-05,Sampaio Corrêa,1,4,São Caetano,VV,0,3,5,1500.00,1500.00,1483.52,1516.48
2,CBR,2003,2003-02-05,Coríntians-RN,2,0,Santa Cruz,VC,3,0,14,1500.00,1500.00,1509.18,1490.82
3,CBR,2003,2003-02-05,CFZ-DF,0,0,Fortaleza,E,1,1,11,1500.00,1500.00,1496.50,1503.50
4,CBR,2003,2003-02-05,Ipatinga,2,1,Goiás,VC,3,0,2,1500.00,1500.00,1509.00,1491.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22652,Série A,2023,2023-08-27,América-MG,2,1,São Paulo,VC,3,0,204,1661.79,1795.51,1683.74,1773.56
22653,Série A,2023,2023-08-27,Grêmio,3,0,Cruzeiro,VC,3,0,202,1740.94,1693.69,1751.25,1683.38
22654,Série A,2023,2023-08-27,Fortaleza,3,1,Coritiba,VC,3,0,201,1796.95,1633.78,1804.69,1626.04
22655,Série A,2023,2023-08-27,Atlético-MG,2,0,Santos,VC,3,0,203,1766.07,1696.78,1775.87,1686.98


In [5]:
elo_df

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
0,Botafogo,1919.20,858,1.0,1667.25,12.0,1167,344,287,227,A
1,Palmeiras,1898.90,849,2.0,1734.05,2.0,1308,407,231,211,A
2,Flamengo,1877.06,885,3.0,1733.82,3.0,1306,409,241,235,A
3,Fluminense,1834.26,879,4.0,1691.46,10.0,1232,362,292,225,A
4,Fortaleza,1804.69,716,5.0,1586.32,24.0,964,280,245,191,A
...,...,...,...,...,...,...,...,...,...,...,...
411,Campinense,1402.73,150,130.0,1456.07,114.0,171,46,68,36,-
412,Fast Clube,1400.57,47,131.0,1430.59,129.0,56,14,23,10,-
414,Mogi Mirim,1389.86,173,132.0,1425.43,131.0,184,47,76,50,-
416,Guaratinguetá,1380.27,240,133.0,1445.94,124.0,290,80,105,55,-
